In [ ]:
import os
import sys
import torch
sys.path.append('../')

In [ ]:
from models import *
from data import TweetDataset, tweet_collate_fn


In [ ]:
def performance(output, mask, ori_tok, sel_tok, sent):
    def jaccard(str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    # For each record
    jac = 0.0
    pred_toks = []
    for idx in range(output.size(0)):
        # Step 1, extract pred_tok from origin_tok
        temp = torch.argmax(output[idx], dim=1)
        temp[mask[idx]==0] = 0
        start = 0
        end = temp.size(0) - 1
        while start <= end and temp[start].item() != 1:
            start += 1
        while start <= end and temp[end].item() != 1:
            end -= 1
        # minus the offset
        start = max(0, start - 1)
        end = min(max(0, end - 1), len(ori_tok[idx])-1)
        if sent[idx].item() == 0 or len(ori_tok) <= 3:
            pred_tok = ori_tok[idx]
        else:
            pred_tok = ori_tok[idx][start:end+1]
        # Step 2, cal jaccard
        jac += jaccard(' '.join(pred_tok), ' '.join(sel_tok[idx]))
        pred_toks.append(pred_tok)
    return jac / output.size(0), pred_toks

In [ ]:
model = create()
model = model.cuda()

In [ ]:
from torch.utils.data import DataLoader
train_data = TweetDataset('/home/liu/DL_workstation/tweet-sent/data/train.csv',
                         64)
loader = DataLoader(train_data, 32, collate_fn = tweet_collate_fn)

In [ ]:
for batch_idx, (text, mask, sel_label, label, ori_tok, sel_tok) in enumerate(loader):
    text = text.cuda()
    mask = mask.cuda()
    out = model(text, mask)
    _, _ = performance(out, mask, ori_tok, sel_tok, label)